# 상관관계 
'06_corr.ipynb'

## 정의
- 두 변수간에 서로 얼마나 함께 변하는지 정도를 나타내는 통계 개념
- 값의 범위는 : -1 ~+1
    - +1: 완벽한 양의 상관관계 => x가 증가하면 y도 비례해서 증가: 키- 몸무게 
    - : 상관없음 : 키와 수학점수 
    - -1 : 완벽한 음의 상관관계 => x가 증가하면 y가 비례하여 감소 : 운동량 - 체지방 

## 상관 계수  ($r$)
두 변수간의 관계가 얼마나 강한지 측정하는 법
- 피어슨 상관계수(Pearson) 상관계수 : 가장 많이 쓰임, 연속 변수 선형관계 측정 
- 스피어만 (SPesarman): 순위 (랭크) 비선형 관계 이상치에 강함 차이가 아무리 나더라도 보기 좋게 등수 표현(우사인 볼트는 아무리 /빨라도 1등 거북이는 아무리 느려도 100등)
- 켄달(Kendal) : 순위 일관성 기반, 표본이 적을 때 안정적 

In [ ]:
%pip install "kagglehub[pandas-datasets]"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Mac OS 기본 한글 폰트 설정
plt.rc('font', family='AppleGothic') 

# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv('./OnlineRetail.csv', encoding='ISO-8859-1')


In [ ]:
from da_utils import profile, outliers, pattern
profile.get_data_profile(df)


In [ ]:
pattern.analyze_missing_patterns(df) #결측이 어떤 패턴으로 나타나는 지 분석하는 함수 

In [ ]:
outliers.outlier_detection(df, 0.999, iso_cont='auto', final_threshold=2)

In [ ]:
df

In [ ]:
print('전체 거래', len(df))
print('고유 InvoiceNo', df['InvoiceNo'].nunique())
print('고유 고객수 ',df['CustomerID'].nunique())

In [ ]:
#데이터 전처리
df_clean = df.copy()

#CustomerID 결측값제거( 고객 단위 분석을 위해 필수) 
missing_customers = df_clean['CustomerID'].isnull().sum()
print(f'CustomerID 결측값 제거: {missing_customers}건')
df_clean = df_clean.dropna(subset=['CustomerID'])

# 취소 거래 분리 (InvoiceNo 가'C'로 시작)
cancel_mask = df_clean['InvoiceNo'].astype('str').str.startswith('C')
df_cancel= df_clean[cancel_mask]
df_clean =df_clean[~cancel_mask]
print(f'취소거래 분리: {len(df_cancel)}건')
print(f' 정상거래 분석: {len(df_clean)}건')

# 파생 변수 생성( 상관 관계 분석): 기존 데이터에서 새로운 변수(컬럼) 생성해서 모델 성능과 분석력 향상 
# TotalAmount, InvoiceDate(DateTime), Year, Month, DayOfWeek(숫자), Hour, DayName(글자), CustomerID(Str)


# TotalAmount 계산 (Quantity × UnitPrice)
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

# InvoiceDate를 datetime 형식으로 변환
df_clean['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 날짜·시간 관련 파생변수 생성
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.weekday  # 월=0, 일=6
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['DayName'] = df_clean['InvoiceDate'].dt.day_name()

# CustomerID를 문자열(str)로 변환
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int).astype(str)

print('\n == 이상값 확인=== ')
print(f'음수 수량: {(df_clean['Quantity'] < 0).sum()}')
print(f'음수 단가: {(df_clean['UnitPrice'] < 0).sum()}')
print(f'0 단가: {(df_clean['UnitPrice'] == 0).sum()}')

# 양수 수량 & 양수 단가 데이터만 살리기
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)]

df_clean.info()


df_clean

In [ ]:

print('=== 고객별 구매 패턴 특성 DF 생성 ===')

customer_stats = df_clean.groupby('CustomerID').agg({
    'InvoiceNo':  'nunique',  # 구매 횟수(Frequency)
    'Quantity': ['sum', 'mean'],  # 총구매량, 평균구매량
    'UnitPrice': 'mean',  # 평균 단가
    'TotalAmount': ['sum', 'mean'],  # 총 구매액(Monetary), 평균 구매액
    'StockCode': 'nunique',  # 구매한 상품 종류 수
    'InvoiceDate': ['min', 'max'],  # 첫 구매일, 마지막 구매일
}).round(2)

# 컬럼명 정리 (구매횟수==F, 총구매액==M)
customer_stats.columns = ['Frequency', '총구매량', '평균구매량', '평균단가', 
                        'Monetary', '평균구매액', '상품종류수', '첫구매일', '마지막구매일']

# 구매 기간 및 주기 계산 (추가하기)
customer_stats['구매기간일수'] = (customer_stats['마지막구매일'] - customer_stats['첫구매일']).dt.days + 1
customer_stats['구매주기'] = customer_stats['구매기간일수'] / customer_stats['Frequency']

# RFM 분석 변수
analysis_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
customer_stats['Recency'] = (analysis_date - customer_stats['마지막구매일']).dt.days


# RFM점수 (모두 5등급) (.qcut으로 5등분)
customer_stats['R_score'] = pd.qcut(customer_stats['Recency'], 5, labels=[5,4,3,2,1])
customer_stats['F_score'] = pd.qcut(customer_stats['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
customer_stats['M_score'] = pd.qcut(customer_stats['Monetary'], 5, labels=[1,2,3,4,5])
customer_stats['RFM_score'] = (customer_stats['R_score'].astype(str) + \
                                customer_stats['F_score'].astype(str) + \
                                customer_stats['M_score'].astype(str))
# 추가정보
customer_stats['평균장바구니크기'] = customer_stats['총구매량'] / customer_stats['Frequency']
customer_stats['거래당상품종류'] = customer_stats['상품종류수'] / customer_stats['Frequency']

# 가격 민감도 - 단가가 상대적으로 일관되게 유지되는가?
# CV = Coefficient of Variation (표준편차 / 평균) => 높으면, 다양한 가격대로 산다 / 낮으면, 특정 가격대에 고정적 구매
price_cv = df_clean.groupby('CustomerID')['UnitPrice'].agg(['mean', 'std'])
price_cv['가격변동성'] = (price_cv['std'] / price_cv['mean']).fillna(0)

customer_stats = customer_stats.merge(
    price_cv[['가격변동성']], # 두 DF 다 컬럼이 아닌 인덱스가 CustomerID
    left_index=True,   # customer_stats의 인덱스 기준으로 병합
    right_index=True   # price_cv의 인덱스 기준으로 병합
)

customer_stats

In [ ]:
df_clean

In [ ]:
print('=== 기본 상관관계 분석 ===')

numeric_cols =  ['Frequency', '총구매량', '평균구매량', '평균단가', 'Monetary', 
                   '평균구매액', '상품종류수', '구매기간일수', '구매주기', 'Recency', '평균장바구니크기', '거래당상품종류', '가격변동성']


correlation_data= customer_stats[numeric_cols]
correlation_data.head(3)

In [ ]:
# 피어슨 상관관계 - 키<->몸무게 / 온도<->전력 같이 실제 값이 비례하는 경우 (값)
pearson_corr = correlation_data.corr(method='pearson')
# 스피어만 상관관계 - 시험순위<->대회순위 / 만족도<->재구매의사 같이 서열형 관계에 적합 (순서)
spearman_corr = correlation_data.corr(method='spearman')

fig, axes = plt.subplots(2, 2, figsize=(20, 16))
a1, a2, a3, a4 = axes[0,0], axes[0,1], axes[1,0], axes[1,1]

# 피어슨 상관관계 히트맵    
sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a1, cbar_kws={'label': 'Pearson 상관계수'})
a1.set_title('피어슨 상관관계 (선형 관계)', fontsize=14)

# 스피어만 상관관계 히트맵
sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a2, cbar_kws={'label': 'Spearman 상관계수'})
a2.set_title('스피어만 상관관계 (순위 기반)', fontsize=14)
# 비선형적인 지표 스피어만으로 바꾸면 선형이라고 바뀌기도함 곡률이 심한 애들일 수록 피어슨은 선형이라고 인정을 안하지만 스피어만은 선형이라고  말하는 경우가 많음  진한 색깔이 그것을의미. 평균구매액과 평균장바구니 크기는 피어슨 상관관계가 더높음 
corr_diff = abs(spearman_corr - pearson_corr)
sns.heatmap(corr_diff, annot=True, fmt='.2f', cmap='Reds',
             square=True, ax=a3, cbar_kws={'label': '|차이|'})
a3.set_title('피어슨 vs 스피어만 차이(비선형성지표)')

# 강한 상관관계 (|r| > 0.5) 네트워크
strong_corr = pearson_corr.copy()
strong_corr[abs(strong_corr) < 0.5] = 0

np.fill_diagonal(strong_corr.values, 0) # numpy 는 판다스의 데이터 프레임을 못 쓰는데 values로 처리 
sns.heatmap(strong_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=a4, cbar_kws={'label': '강한 상관관계'})
a4.set_title('강한 상관관계 (|r| > 0.5)', fontsize=14)
    
plt.tight_layout()
plt.show()

#스피어만 상관계수를 사용 하는 경우: 총 구매액 은 구매량이 올라면 올라간다!  총구매액은 무조건 총구매량의 영향을 받을수 밖에 없다. 
# 장바구니 크기의 1,2,3,4  평균구매액도 순위로 치환해서 보니까 (한번 구매량) ,피어슨이만이 기본이다. 순위기반은 특수케이스다. 순위의 상승을 보고 싶냐 숫자의 상승을 보고 싶냐의 차이다!  켄달타우(순위기반)

#주요발견 사항 요약
print('\n ===주요 상관관계 발견사항===')
# 아래 히트맵에서 위쪽 삼각형만 보겠다. 
upper_triangle = np.triu(pearson_corr, k=1)
# 가장 큰 값이 있는 인덱스
strong_positive = np.argmax(upper_triangle)

#가장 강한 양의 상관관계 
# max_corr  = pearson_corr.iloc[strong_positive]
# print(f'가장 강한 양의 상관관계')
# print(f'  {pearson_corr.index[strong_positive[0]]} <-> {pearson_corr.index[strong_positive[0]]})








In [85]:
print('매출 증대 핵심 요인')
monetary_corr = pearson_corr['Monetary'].abs().sort_values(ascending=False)

for factor, corr in monetary_corr.head(6).items():
    if factor != 'Monetary':
        print(f' {factor}: {corr:3f}')



#print('고객 충성도 관련 요인')
monetary_corr = pearson_corr['Frequency'].abs().sort_values(ascending=False)

for factor, corr in monetary_corr.head(6).items():
    if factor != 'Monetary':
        print(f' {factor}: {corr:.3f}')

매출 증대 핵심 요인
 총구매량: 0.922907
 Frequency: 0.553650
 상품종류수: 0.386977
 평균장바구니크기: 0.310360
 평균구매액: 0.287264
 Frequency: 1.000
 상품종류수: 0.692
 총구매량: 0.558
 구매기간일수: 0.477
 Recency: 0.261
